In [2]:
# default_exp tef_core
from nbdev import *

# TEF core framework

> contains TEF framework

In [3]:
#export
import math
import numpy as np
import xarray as xr
import time

In [5]:
#export

data_description = {
    "lon" : "xt_ocean",
    "lat" : "yt_ocean",
    "depth" : "st_ocean",
    "time" : "time"
}

class constructorTEF:
    
    def __init__(self, inputFile, data_description, **kwargs):
        if isinstance(inputFile, str):
            try:
                self._read(fileName=inputFile)
            except (OSError, IOError, RunetimeError):
                raise IOError("Could not read file.")
        elif isinstance(inputFile, xr.Dataset):
            self.ds = inputFile
        elif isinstance(inputFile, xr.DataArray):
            self.ds = inputFile
        else:
            raise IOError("Please provide xarray Dataset/DataArray or netcdf-filepath")
        
        self._setup(data=data_description)
        
        self.transport = None
        self.tracer = None
        
    def _read(self, fileName, **kwargs):
        self.ds = xr.open_dataset(filename, use_cftime=True, **kwargs)
        
    def _setup(self, data):
        
        for dimension in data.keys():
            if data[dimension] is None:
                self.ds = self.ds.expand_dims(dimension)
            else:
                self.ds = self.ds.rename({data[dimension]: dimension})
                                                       
        self.ds = self.ds.transpose("time",
                                    "depth",
                                    "lat",
                                    "lon")

In [4]:
show_doc(constructorTEF)

NameError: name 'show_doc' is not defined

The user can either provide the path to a netCDF-dataset using `filename`, which will then be loaded as a xarray dataset. If a pre-loaded xarray dataset is already present, the user can simply assign this to the `TEF_object` by using `ds`.

In [6]:
show_doc(TEF_object.timesteps)

<h4 id="TEF_object.timesteps" class="doc_header"><code>TEF_object.timesteps</code><a href="" class="source_link" style="float:right">[source]</a></h4>



Returns the number of timesteps of the given dataset.

In [7]:
show_doc(TEF_object.read)

<h4 id="TEF_object.read" class="doc_header"><code>TEF_object.read</code><a href="__main__.py#L68" class="source_link" style="float:right">[source]</a></h4>

> <code>TEF_object.read</code>(**`filename`**, **\*\*`kwargs`**)

Reads xarray dataset

Is called during the initialization of the `TEF_object` when a `filename` is provided.

In [8]:
show_doc(TEF_object.set_up)

<h4 id="TEF_object.set_up" class="doc_header"><code>TEF_object.set_up</code><a href="__main__.py#L76" class="source_link" style="float:right">[source]</a></h4>

> <code>TEF_object.set_up</code>(**`time_name`**=*`None`*, **`longitude_name`**=*`None`*, **`latitude_name`**=*`None`*, **`depth_name`**=*`None`*)

Sets up a xarray.dataset and transposes dimensions into needed format

`TEF_object.set_up` tries to automatically find the corresponding dimensions. If no 4D array is provided, dummy dimensions will be created. However, sometimes the naming of the dimensions deviates from common dimension descriptions. If so, the explicit dimension names can be provided as well. The function automatically reorders the dimensions into `time, z, y, x`.

In [9]:
# Try using uncommon dimensions
import xarray as xr
import numpy as np 

lat = np.arange(-180, 180)
lon = 37
depth = np.arange(0, 200)
time = 2
dummy_var = np.zeros((lat.size, depth.size))

ds = xr.Dataset({'dummy': (["lAtiTude", "dePht"], dummy_var)},
                coords = {'lAtiTude': (["lAtiTude"], lat),
                          'dePht': (["dePht"], depth)})

tef = TEF_object(ds=ds)
tef.set_up(latitude_name="lAtiTude", depth_name="dePht")
tef.ds

Warning: 
 'time' dimension (dtype='datetime64[ns]') not found.

In [ ]:
show_doc(TEF_object._get_name_time)

<h4 id="TEF_object._get_name_time" class="doc_header"><code>TEF_object._get_name_time</code><a href="__main__.py#L129" class="source_link" style="float:right">[source]</a></h4>

> <code>TEF_object._get_name_time</code>(**`x`**=*`None`*)

check for 'time' dimension and return name

In [ ]:
show_doc(TEF_object._get_name_depth)

<h4 id="TEF_object._get_name_depth" class="doc_header"><code>TEF_object._get_name_depth</code><a href="__main__.py#L191" class="source_link" style="float:right">[source]</a></h4>

> <code>TEF_object._get_name_depth</code>()

check for 'depth' dimension and return name

In [ ]:
show_doc(TEF_object._get_name_longitude)

<h4 id="TEF_object._get_name_longitude" class="doc_header"><code>TEF_object._get_name_longitude</code><a href="__main__.py#L160" class="source_link" style="float:right">[source]</a></h4>

> <code>TEF_object._get_name_longitude</code>()

check for 'longitude' dimension and return name

In [ ]:
show_doc(TEF_object._get_name_latitude)

<h4 id="TEF_object._get_name_latitude" class="doc_header"><code>TEF_object._get_name_latitude</code><a href="__main__.py#L176" class="source_link" style="float:right">[source]</a></h4>

> <code>TEF_object._get_name_latitude</code>()

check for 'latitude' dimension and return name